ModuleNotFoundError: No module named 'numpy'

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:

import csv

# File paths
video = 'hip_hop_1.mp4'
csv_file = 'hip_hop_1.csv'

# Initialize Mediapipe holistic model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Open video file
cap = cv2.VideoCapture(video)

# Define the scale for resizing
scale_percent = 50  # Resize to 50% of the original size

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Calculate new dimensions
new_width = int(frame_width * scale_percent / 100)
new_height = int(frame_height * scale_percent / 100)

# Open CSV file for writing
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row for the CSV file
    header = ['frame'] + [f'{landmark}_{coord}' for landmark in range(33) for coord in ('x', 'y', 'z', 'visibility')]
    writer.writerow(header)

    # Initialize holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        frame_index = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Resize frame
            frame = cv2.resize(frame, (new_width, new_height))

            # Recolor feed
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detections
            results = holistic.process(image)

            # Recolor image back to BGR for rendering
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Check for pose landmarks
            if results.pose_landmarks:
                # Extract pose landmarks
                pose_landmarks = results.pose_landmarks.landmark
                landmarks = [frame_index] + [coord for landmark in pose_landmarks for coord in (landmark.x, landmark.y, landmark.z, landmark.visibility)]
                writer.writerow(landmarks)

                # Draw pose landmarks on the frame
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

            # Display the frame
            cv2.imshow('Image', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

            frame_index += 1

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV data
csv_file = 'hip_hop_1.csv'
data = pd.read_csv(csv_file)

# Function to plot multiple landmarks over frames in four separate graphs
def plot_landmarks_separately(data, landmarks_indices):
    frames = data['frame']
    fig, axs = plt.subplots(4, 1, figsize=(14, 20), sharex=True)

    for landmark_index in landmarks_indices:
        x = data[f'{landmark_index}_x']
        y = data[f'{landmark_index}_y']
        z = data[f'{landmark_index}_z']
        visibility = data[f'{landmark_index}_visibility']
        
        axs[0].plot(frames, x, label=f'Landmark {landmark_index} X')
        axs[1].plot(frames, y, label=f'Landmark {landmark_index} Y')
        axs[2].plot(frames, z, label=f'Landmark {landmark_index} Z')
        axs[3].plot(frames, visibility, label=f'Landmark {landmark_index} Visibility')

    axs[0].set_ylabel('X Coordinate')
    axs[1].set_ylabel('Y Coordinate')
    axs[2].set_ylabel('Z Coordinate')
    axs[3].set_ylabel('Visibility')
    
    axs[3].set_xlabel('Frame')
    
    axs[0].set_title('X Coordinates Over Frames')
    axs[1].set_title('Y Coordinates Over Frames')
    axs[2].set_title('Z Coordinates Over Frames')
    axs[3].set_title('Visibility Over Frames')
    
    for ax in axs:
        ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1))
    
    plt.tight_layout()
    plt.show()

# List of landmarks to plot
landmarks_to_plot = [25, 26, 27, 28]

# Plot the specified landmarks
plot_landmarks_separately(data, landmarks_to_plot)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Load the CSV data
csv_file = 'hip_hop_1.csv'
data = pd.read_csv(csv_file)

# Function to plot multiple landmarks over frames in four separate graphs
def plot_landmarks_separately(data, landmarks_indices):
    frames = data['frame']
    fig, axs = plt.subplots(4, 1, figsize=(14, 20), sharex=True)

    for landmark_index in landmarks_indices:
        x = data[f'{landmark_index}_x']
        y = data[f'{landmark_index}_y']
        z = data[f'{landmark_index}_z']
        visibility = data[f'{landmark_index}_visibility']
        
        axs[0].plot(frames, x, label=f'Landmark {landmark_index} X')
        axs[1].plot(frames, y, label=f'Landmark {landmark_index} Y')
        axs[2].plot(frames, z, label=f'Landmark {landmark_index} Z')
        axs[3].plot(frames, visibility, label=f'Landmark {landmark_index} Visibility')

    axs[0].set_ylabel('X Coordinate')
    axs[1].set_ylabel('Y Coordinate')
    axs[2].set_ylabel('Z Coordinate')
    axs[3].set_ylabel('Visibility')
    
    axs[3].set_xlabel('Frame')
    
    axs[0].set_title('X Coordinates Over Frames')
    axs[1].set_title('Y Coordinates Over Frames')
    axs[2].set_title('Z Coordinates Over Frames')
    axs[3].set_title('Visibility Over Frames')
    
    for ax in axs:
        ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1))
    
    plt.tight_layout()
    plt.show()

# List of landmarks to plot
landmarks_to_plot = [25, 26, 27, 28]

# Plot the specified landmarks
plot_landmarks_separately(data, landmarks_to_plot)


In [ ]:
def detect_peaks(data, landmarks_indices, height=None, distance=None):
    peaks = {}
    frames = data['frame']
    
    for landmark_index in landmarks_indices:
        peaks[landmark_index] = {}
        x = data[f'{landmark_index}_x']
        y = data[f'{landmark_index}_y']
        z = data[f'{landmark_index}_z']
        
        peaks[landmark_index]['x_peaks'], _ = find_peaks(x, height=height, distance=distance)
        peaks[landmark_index]['y_peaks'], _ = find_peaks(y, height=height, distance=distance)
        peaks[landmark_index]['z_peaks'], _ = find_peaks(z, height=height, distance=distance)
        
    return peaks

# Detect peaks for the specified landmarks
peaks = detect_peaks(data, landmarks_to_plot, height=0.5, distance=10)

# Print detected peaks for visualization
for landmark_index in landmarks_to_plot:
    print(f'Landmark {landmark_index} Peaks:')
    print(f"X peaks at frames: {peaks[landmark_index]['x_peaks']}")
    print(f"Y peaks at frames: {peaks[landmark_index]['y_peaks']}")
    print(f"Z peaks at frames: {peaks[landmark_index]['z_peaks']}")


In [ ]:
def analyze_movements(data, peaks, landmarks_indices):
    frames = data['frame']
    
    for landmark_index in landmarks_indices:
        x_peaks = peaks[landmark_index]['x_peaks']
        y_peaks = peaks[landmark_index]['y_peaks']
        z_peaks = peaks[landmark_index]['z_peaks']
        
        print(f'Landmark {landmark_index} Analysis:')
        print(f"Detected {len(x_peaks)} peaks in X coordinates.")
        print(f"Detected {len(y_peaks)} peaks in Y coordinates.")
        print(f"Detected {len(z_peaks)} peaks in Z coordinates.")
        
        plt.figure(figsize=(14, 6))
        plt.plot(frames, data[f'{landmark_index}_x'], label=f'Landmark {landmark_index} X')
        plt.plot(x_peaks, data[f'{landmark_index}_x'][x_peaks], "x")
        plt.plot(frames, data[f'{landmark_index}_y'], label=f'Landmark {landmark_index} Y')
        plt.plot(y_peaks, data[f'{landmark_index}_y'][y_peaks], "x")
        plt.plot(frames, data[f'{landmark_index}_z'], label=f'Landmark {landmark_index} Z')
        plt.plot(z_peaks, data[f'{landmark_index}_z'][z_peaks], "x")
        plt.xlabel('Frame')
        plt.ylabel('Value')
        plt.title(f'Landmark {landmark_index} Over Frames with Detected Peaks')
        plt.legend()
        plt.show()

# Analyze movements for the specified landmarks
analyze_movements(data, peaks, landmarks_to_plot)
